In [1]:
import os
import pandas as pd
import pymysql
import numpy as np

def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# KFOOD메뉴데이터

# RT_KFD_MENU_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=4d954cb0-3b98-11eb-af9a-4b03f0a582d6

In [6]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_KFD_MENU_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [7]:
with dev_db_conn() as conn:
    sql = """
    SELECT DISTINCT T4.PRODUCT_ID AS 'MENU_ID', T4.MENU_NM, T4.MENU_PRICE AS 'MENU_PRC', 'KRW' AS 'CRNCY_CD', 'KOR' AS 'COUNTRY_NM', 
        REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
    T3.SRC_ID AS 'RSTRNT_ID'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    INNER JOIN redtable01.MENU T4 ON T3.RSTR_ID = T4.RSTR_ID
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
    AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%？%' AND T4.PRODUCT_ID IS NOT NULL AND T4.PRODUCT_ID<>''
    GROUP BY T4.PRODUCT_ID;
    """
    df = pd.read_sql(sql, conn)
    
# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

In [8]:
df.columns

Index(['MENU_ID', 'MENU_NM', 'MENU_PRC', 'CRNCY_CD', 'COUNTRY_NM', 'CTY_NM',
       'RSTRNT_ID'],
      dtype='object')

In [9]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_KFD_MENU_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["MENU_ID"], row["MENU_NM"], row["MENU_PRC"], row["CRNCY_CD"],
                                  row["COUNTRY_NM"], row["CTY_NM"], row["RSTRNT_ID"]), axis=1).tolist()
        curs.executemany(sql, val)